 # MOVIE RECOMMENDATION ENGINE


**MOVIE RECOMMENDATION ENGINE** : A Engine which will recommend best movie to the user

Importing library files
1. Pandas
1. Numpy
1. Matplotlib
1. Seaborn

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'numpy'

**Load the dataset through pandas library**

In [ ]:
movies=pd.read_csv('../input/mr-data/movies.csv')  # movies data
ratings= pd.read_csv('../input/mr-data/ratings.csv')  # ratings data
movies.head(2) # movies dataset visualization

: 

In [ ]:
ratings.head(2)  # ratings dataset visualization

: 

**Find all types of  unique movie genres**

In [ ]:
movies['genres'].unique()

: 

**Find all types of unique ratings of movies**

In [ ]:
ratings['rating'].unique()

: 

**Find all types of unique movie titles**

In [ ]:
movies['title'].unique()

: 

**Countplot of user ratings**

In [ ]:
sns.countplot(ratings['rating'])

: 

Pie Plot of Movies ratings

In [ ]:
plt.figure(figsize=(10,8))
ratings['rating'].value_counts().plot.pie(autopct='%1.1f%%',shadow=True)
plt.show()

: 

In [ ]:
ratings=ratings[['userId','movieId','rating']] #separate columns from timestamp column
ratings_df=ratings.groupby(['userId','movieId']).agg(np.max) #Groupby data
ratings_df.head()

: 

**On the above, groupby data based on : Each user viewed which movies and how much rated for those movies.
userid 1 viewed movie ids 1,3,6,47,50 ans so on and gave individual rating for each movie.**

In [ ]:
count_ratings=ratings.groupby('rating').count() #count all ratings
count_ratings

: 

In [ ]:
count_ratings['perc_total']=round(count_ratings['userId']*100/count_ratings['userId'].sum(),
                                  1)
count_ratings

: 

**On the above predict the percentage for each ratings. 
1. ratings 5.0 got 13.1 %
1. ratings 4.5 got 8.5 %
1. ratings 4.0 got 26.6 %
1. ratings 3.5 got 13.0 %
1. ratings 3.0 got 19.9 %
1. ratings 2.5 got 5.5 %
1. ratings 2.0 got 7.5 %
1. ratings 1.5 got 1.8 %
1. ratings 1.0 got 2.8 %
1. ratings 0.5 got 1.4 %
Highest rating is 4.0 and Lowest rating is 1.4 &**

In [ ]:
# PLotting of each ratings
count_ratings['perc_total'].plot.pie()  #pie plot

: 

In [ ]:
count_ratings['perc_total'].plot.bar()  # bar plot

: 

In [ ]:
genres=movies['genres']
genres.head()  # visualization of genres column from movie dataset

: 

In [ ]:
genre_list=" "
for index,row in movies.iterrows():
    genre_list+=row.genres+"|"
genre_list_split=genre_list.split("|")
new_list=list(set(genre_list_split))
new_list.remove('')
new_list

: 

**On the above split the all movie genres from the dataset and used "set" data structure for non duplication of genres in the same. 
If in the column "|" finds, it will separate the genres of movies and remove the duplication in the same by using "set" data structure.**

In [ ]:
m=movies.copy() #make a copy of movies dataset

: 

In [ ]:
for genre in new_list:
    m[genre]=m.apply(lambda _:int(genre in _.genres),axis=1)
m.head()

: 

**Make a Data with title of the movie and genre of the data. If the genre of the movie matches the entire column then it shows 1.
Like movie id 3 whose title is Grumpier Old Men(1995) and genre is Comedy|Romance. This movies genre matches the Romance column very much. '...' indicates there are few columns.**

In [ ]:
avg=pd.DataFrame(ratings.groupby('movieId')['rating'].agg(['mean','count']))
avg # make a dataframe for each movie id with its corresponding ratings mean and count

: 

In [ ]:
avg['movieId']=avg.index
avg # add the movieid column

: 

In [ ]:
np.percentile(avg['count'],70)

: 

**On the above 70 % movie has average count 7**

In [ ]:
np.percentile(avg['count'],50)

: 

**On the above 50 % movie has average count 3**

In [ ]:
idx2title={int(row['movieId']):row['title']
          for _,row in movies.iterrows()}
idx2title

: 

**On the above display the all movie titles arranged one after the another**

In [ ]:
title2idx={j:i for i,j in idx2title.items()}
title2idx

: 

**On the above display the all movie titles arranged one after the another in reverse order**

In [ ]:
highratings=ratings[ratings.rating>=4]
highratings

: 

**Display the movie data whose ratings is greater than or equal to 4**

In [ ]:
itemsets=[[idx2title[mov] for mov in highratings[highratings.userId==user].movieId]
         for user in highratings.userId]
itemsets

: 

**On the above display all movie names whose rating is greater than or equal to 4.0**

# Recommendation Engine making starts here


**Recommendation engine will be made by using Apriori algorithm which is very much popular algorithm for Association Rule Mining.**

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
te=TransactionEncoder()
tr_ary=te.fit(itemsets).transform(itemsets)
DF=pd.DataFrame(tr_ary,columns=te.columns_)
DF.head()

: 

**On the above rows has been converted into columns and columns has been converted into rows.
Movie does not rated is False and rated is True.**

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
f=apriori(DF, min_support=0.2, use_colnames=True, max_len=2)
rules=association_rules(f,metric='lift',min_threshold=2)
rules.head()

: 

**On the above, the movie has been recommended to use user where 'antecedents' is the movie column and 'consequents' is the recommended movie column.
In the first row if the see the '(2001:A Space Odyssey(1968))' movie then '(Blade Runner(1982))' will be recommended to you for further view and whose confience of the view is 72 % approximately.
Lift column is used for to tell us likelihood of watching both movies together is 2.144151 times more than the likelihood of just watching one movie.
Support is the default popularity of an item. In mathematical terms the support of one item is nothing but the ratio of transactions involving one item to the total number of transactions. 
support(movie_1)=(All transactions involving movie_1)/(total transactions)
Conviction compares the probability that X appears without Y if they were dependent with the actual frequency of the appearance of X without Y.
Threshold in the Apriori algorithm identifies the item sets which are subsets of at least as each transaction is seen as a set of items.**